In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import collections
from sklearn import preprocessing
from sklearn import metrics
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from scipy import stats
import seaborn as sns

In [0]:
from sklearn import tree

In [0]:
train_data = pd.read_csv('./drive/My Drive/BT4240 data/result.csv')\
              .drop(columns=['Unnamed: 0', 'Unnamed: 0.1'],axis=1)
train_data.head()

,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved,description,total_cost,Project Resource Category,Project Need Statement
0,p039565,df72a3ba8089423fa8a94be88060f6ed,Mrs.,GA,2017-04-26 15:57:28,Grades 3-5,"Music & The Arts, Health & Sports","Performing Arts, Team Sports",Keep Calm and Dance On,Our elementary school is a culturally rich sch...,We strive to provide our diverse population of...,NaN,NaN,My students need matching shirts to wear for d...,1,0,{Reebok Girls' Fashion Dance Graphic T-Shirt -...,400.00,Supplies,My students need matching shirts to wear for d...
1,p063374,403c6783e9286e51ab318fba40f8d729,Mrs.,DE,2016-11-05 10:01:51,Grades PreK-2,"Applied Learning, Literacy & Language","Character Education, Literature & Writing",Need to Reach Our Virtual Mentors!!!,My kids tell me each day that they want to mak...,I started a program called Telementoring in ho...,NaN,NaN,My students need tablets in order to communic...,0,1,{Amazon - Fire Kids Edition - 7'- Tablet - 16G...,727.36,Technology,My students need tablets in order to communic...
2,p114989,b580c11b1497a0a67317763b7f03eb27,Ms.,IN,2016-09-13 22:35:57,Grades 6-8,Math & Science,Mathematics,Wobble Chairs Help Fidgety Kids Focus,My seventh graders dream big. They can't wait ...,I have used alternative seating in my classroo...,NaN,NaN,My students need seating that allows the most ...,13,1,"{Kore Patented WOBBLE Chair, Made in the USA, ...",319.80,Supplies,My students need seating that allows the most ...
3,p030093,b9e731e16ad8669f37a43e5316518106,Teacher,VA,2017-03-09 17:16:13,Grades PreK-2,"Literacy & Language, Math & Science","Literature & Writing, Mathematics","Writing, Writing, Writing!",My classroom has 24 students in it. We have an...,"In our classroom, we use dry erase boards thro...",NaN,NaN,My students need individual dry erase boards t...,1,1,{BOARD LEARNING THE CLASSICS W/MARKER},102.00,Supplies,My students need individual dry erase boards t...
4,p225747,4681d60136ea859084ab488f2254b219,Ms.,PA,2016-12-08 13:25:03,Grades 3-5,"Applied Learning, Literacy & Language","Character Education, Literacy",Life Lessons through Literacy,"My school is located in a high poverty area, j...",As a Reading Specialist who serves Grades 1-8 ...,NaN,NaN,My students need books that teach life lessons...,3,1,"{A Bad Case of Stripes (Scholastic Bookshelf),...",176.59,Books,My students need books that teach life lessons...


In [0]:
train_data['project_subject_categories'] = train_data['project_subject_categories'].str.split(", ").to_frame()
train_data['project_subject_subcategories'] = train_data['project_subject_subcategories'].str.split(", ").to_frame()
train_data['Project Resource Category'] = train_data['Project Resource Category'].str.split(", ").to_frame()

train_data.head(1)

,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved,description,total_cost,Project Resource Category,Project Need Statement
0,p039565,df72a3ba8089423fa8a94be88060f6ed,Mrs.,GA,2017-04-26 15:57:28,Grades 3-5,"[Music & The Arts, Health & Sports]","[Performing Arts, Team Sports]",Keep Calm and Dance On,Our elementary school is a culturally rich sch...,We strive to provide our diverse population of...,NaN,NaN,My students need matching shirts to wear for d...,1,0,{Reebok Girls' Fashion Dance Graphic T-Shirt -...,400.0,[Supplies],My students need matching shirts to wear for d...


In [0]:
train_data['project_subject_categories'] = train_data['project_subject_categories'].apply(collections.Counter)
train_data['project_subject_subcategories'] = train_data['project_subject_subcategories'].apply(collections.Counter)
train_data['Project Resource Category'] = train_data['Project Resource Category'].apply(collections.Counter)

project_subject_categories = pd.DataFrame.from_records(train_data['project_subject_categories']).fillna(value=0).add_prefix("proj_cat_")
project_subject_subcategories = pd.DataFrame.from_records(train_data['project_subject_subcategories']).fillna(value=0).add_prefix("proj_sub_cat_")
Project_Resource_Category = pd.DataFrame.from_records(train_data['Project Resource Category']).fillna(value=0).add_prefix("res_cat_")


train_data = pd.concat([train_data, project_subject_categories], axis=1)
train_data = pd.concat([train_data, project_subject_subcategories], axis=1)
train_data = pd.concat([train_data, Project_Resource_Category], axis=1)

train_data.head(5)

,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved,description,total_cost,Project Resource Category,Project Need Statement,proj_cat_Applied Learning,proj_cat_Care & Hunger,proj_cat_Health & Sports,proj_cat_History & Civics,proj_cat_Literacy & Language,proj_cat_Math & Science,proj_cat_Music & The Arts,proj_cat_Special Needs,proj_cat_Warmth,proj_sub_cat_Applied Sciences,proj_sub_cat_Care & Hunger,proj_sub_cat_Character Education,proj_sub_cat_Civics & Government,proj_sub_cat_College & Career Prep,proj_sub_cat_Community Service,proj_sub_cat_ESL,proj_sub_cat_Early Development,proj_sub_cat_Economics,proj_sub_cat_Environmental Science,proj_sub_cat_Extracurricular,proj_sub_cat_Financial Literacy,proj_sub_cat_Foreign Languages,proj_sub_cat_Gym & Fitness,proj_sub_cat_Health & Life Science,proj_sub_cat_Health & Wellness,proj_sub_cat_History & Geography,proj_sub_cat_Literacy,proj_sub_cat_Literature & Writing,proj_sub_cat_Mathematics,proj_sub_cat_Music,proj_sub_cat_Nutrition Education,proj_sub_cat_Other,proj_sub_cat_Parent Involvement,proj_sub_cat_Performing Arts,proj_sub_cat_Social Sciences,proj_sub_cat_Special Needs,proj_sub_cat_Team Sports,proj_sub_cat_Visual Arts,proj_sub_cat_Warmth,res_cat_Art Supplies,res_cat_Books,res_cat_Classroom Basics,res_cat_Clothing & Hygiene,res_cat_Computers & Tablets,res_cat_Desks & Storage,res_cat_Educational Kits & Games,res_cat_Flexible Seating,res_cat_Food,res_cat_Instructional Technology,res_cat_Lab Equipment,res_cat_Musical Instruments,res_cat_Other,res_cat_Reading Nooks,res_cat_Sports & Exercise Equipment,res_cat_Supplies,res_cat_Technology,res_cat_Visitors
0,p039565,df72a3ba8089423fa8a94be88060f6ed,Mrs.,GA,2017-04-26 15:57:28,Grades 3-5,"{'Music & The Arts': 1, 'Health & Sports': 1}","{'Performing Arts': 1, 'Team Sports': 1}",Keep Calm and Dance On,Our elementary school is a culturally rich sch...,We strive to provide our diverse population of...,NaN,NaN,My students need matching shirts to wear for d...,1,0,{Reebok Girls' Fashion Dance Graphic T-Shirt -...,400.00,{'Supplies': 1},My students need matching shirts to wear for d...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,p063374,403c6783e9286e51ab318fba40f8d729,Mrs.,DE,2016-11-05 10:01:51,Grades PreK-2,"{'Applied Learning': 1, 'Literacy & Language': 1}","{'Character Education': 1, 'Literature & Writi...",Need to Reach Our Virtual Mentors!!!,My kids tell me each day that they want to mak...,I started a program called Telementoring in ho...,NaN,NaN,My students need tablets in order to communic...,0,1,{Amazon - Fire Kids Edition - 7'- Tablet - 16G...,727.36,{'Technology': 1},My students need tablets in order to communic...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,p114989,b580c11b1497a0a67317763b7f03eb27,Ms.,IN,2016-09-13 22:35:57,Grades 6-8,{'Math & Science': 1},{'Mathematics': 1},Wobble Chairs Help Fidgety Kids Focus,My seventh graders dream big. They can't wait ...,I have used alternative seating in my classroo...,NaN,NaN,My students need seating that allows the most ...,13,1,"{Kore Patented WOBBLE Chair, Made in the USA, ...",319.80,{'Supplies': 1},My students need seating that allows the most ...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.

In [0]:
!pip install iteration_utilities
from iteration_utilities import duplicates
list(duplicates(train_data.columns))

     |████████████████████████████████| 112kB 9.7MB/s 
  Created wheel for iteration-utilities: filename=iteration_utilities-0.7.0-cp36-cp36m-linux_x86_64.whl size=189305 sha256=81159c86d06430441e074be93150effd5fa43f263178b34b29789b863391bc9e
  Stored in directory: /root/.cache/pip/wheels/41/ea/fd/de8371ff85ff0aafd560d50a77ca22d107117094d660131e2a
Successfully built iteration-utilities


[]

In [0]:
train_data = train_data.loc[:,~train_data.columns.duplicated()]
train_data = train_data.drop(columns = ['proj_sub_cat_Care & Hunger', 'proj_sub_cat_Special Needs', 'proj_sub_cat_Warmth'])
len(train_data.columns)

74

In [0]:
train_df = train_data.select_dtypes(exclude=['object','category'])
print(len(train_df.columns))
print(train_df.dtypes.unique())
cat_columns = [col for col in train_df.columns if 'cat' in col]
train_df.head(1)

57
[dtype('int64') dtype('float64')]


,teacher_number_of_previously_posted_projects,project_is_approved,total_cost,proj_cat_Applied Learning,proj_cat_Care & Hunger,proj_cat_Health & Sports,proj_cat_History & Civics,proj_cat_Literacy & Language,proj_cat_Math & Science,proj_cat_Music & The Arts,proj_cat_Special Needs,proj_cat_Warmth,proj_sub_cat_Applied Sciences,proj_sub_cat_Character Education,proj_sub_cat_Civics & Government,proj_sub_cat_College & Career Prep,proj_sub_cat_Community Service,proj_sub_cat_ESL,proj_sub_cat_Early Development,proj_sub_cat_Economics,proj_sub_cat_Environmental Science,proj_sub_cat_Extracurricular,proj_sub_cat_Financial Literacy,proj_sub_cat_Foreign Languages,proj_sub_cat_Gym & Fitness,proj_sub_cat_Health & Life Science,proj_sub_cat_Health & Wellness,proj_sub_cat_History & Geography,proj_sub_cat_Literacy,proj_sub_cat_Literature & Writing,proj_sub_cat_Mathematics,proj_sub_cat_Music,proj_sub_cat_Nutrition Education,proj_sub_cat_Other,proj_sub_cat_Parent Involvement,proj_sub_cat_Performing Arts,proj_sub_cat_Social Sciences,proj_sub_cat_Team Sports,proj_sub_cat_Visual Arts,res_cat_Art Supplies,res_cat_Books,res_cat_Classroom Basics,res_cat_Clothing & Hygiene,res_cat_Computers & Tablets,res_cat_Desks & Storage,res_cat_Educational Kits & Games,res_cat_Flexible Seating,res_cat_Food,res_cat_Instructional Technology,res_cat_Lab Equipment,res_cat_Musical Instruments,res_cat_Other,res_cat_Reading Nooks,res_cat_Sports & Exercise Equipment,res_cat_Supplies,res_cat_Technology,res_cat_Visitors
0,1,0,400.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [0]:
train_df[cat_columns] = train_df[cat_columns].astype('int8')
train_df.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


,teacher_number_of_previously_posted_projects,project_is_approved,total_cost,proj_cat_Applied Learning,proj_cat_Care & Hunger,proj_cat_Health & Sports,proj_cat_History & Civics,proj_cat_Literacy & Language,proj_cat_Math & Science,proj_cat_Music & The Arts,proj_cat_Special Needs,proj_cat_Warmth,proj_sub_cat_Applied Sciences,proj_sub_cat_Character Education,proj_sub_cat_Civics & Government,proj_sub_cat_College & Career Prep,proj_sub_cat_Community Service,proj_sub_cat_ESL,proj_sub_cat_Early Development,proj_sub_cat_Economics,proj_sub_cat_Environmental Science,proj_sub_cat_Extracurricular,proj_sub_cat_Financial Literacy,proj_sub_cat_Foreign Languages,proj_sub_cat_Gym & Fitness,proj_sub_cat_Health & Life Science,proj_sub_cat_Health & Wellness,proj_sub_cat_History & Geography,proj_sub_cat_Literacy,proj_sub_cat_Literature & Writing,proj_sub_cat_Mathematics,proj_sub_cat_Music,proj_sub_cat_Nutrition Education,proj_sub_cat_Other,proj_sub_cat_Parent Involvement,proj_sub_cat_Performing Arts,proj_sub_cat_Social Sciences,proj_sub_cat_Team Sports,proj_sub_cat_Visual Arts,res_cat_Art Supplies,res_cat_Books,res_cat_Classroom Basics,res_cat_Clothing & Hygiene,res_cat_Computers & Tablets,res_cat_Desks & Storage,res_cat_Educational Kits & Games,res_cat_Flexible Seating,res_cat_Food,res_cat_Instructional Technology,res_cat_Lab Equipment,res_cat_Musical Instruments,res_cat_Other,res_cat_Reading Nooks,res_cat_Sports & Exercise Equipment,res_cat_Supplies,res_cat_Technology,res_cat_Visitors
0,1,0,400.00,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,0,1,727.36,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,13,1,319.80,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,1,1,102.00,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,3,1,176.59,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
X = train_df.loc[:, train_df.columns != 'project_is_approved']
y = train_df.loc[:, train_df.columns == 'project_is_approved']
print(X.shape)
print(y.shape)

(121308, 56)
(121308, 1)


In [0]:
y['project_is_approved'].value_counts()

1    110358
0     10950
Name: project_is_approved, dtype: int64

In [0]:
clf = tree.DecisionTreeClassifier()

In [0]:
clf = clf.fit(X, y)

In [0]:
tree.plot_tree(clf)